# Simulating Pointing

This is an attempt to simulate pointing errors. There are four frames that descibe the location of an object:

1. the J2000 equatorial frame used by Stellarium
2. the ICRS equatorial frame used by astropy
3. the AltAz frame of the location and time
4. the telecope frame

There is a known, small difference between the first two. So small that we will ignore it for now. The transformation between frames 2 and 3 is handled by astropy. Refraction corrections should be enabled in astropy
to match Stellarium's version. The final transformation, between 3 and 4, requires data from encoders, a pointing model, and an alignment model.

Steps to simulate encoder data:

1. Pick an object above the horizon and select it in Stellarium
2. Using the `StellariumRPC` class, get Stellarium's: RA, Dec, Azi, Alt
3. Have `astropy` compute an Azi and Alt from Stellarium's RA and Dec
4. Simulate alignment using a known rotation matrix
5. Simulate pointing by modeling mount/telescope imperfections
6. Translate into encoder counts

Steps to reconstruct Stellarium data:

1. Need to compute alignment matrix and pointing model
2. Translate from counts to Azi and Alt
3. Have `astropy` compute an RA and Dec from these Azi and Alt
4. Compare to the origonal RA and Dec


In [40]:
import numpy as np
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import EarthLocation, SkyCoord, AltAz
from pushto.stellarium import StellariumRPC
from pushto.util import equatorial_to_horizontal
from pushto.alignment import vec_from_angles, angles_from_vec

In [31]:
rpc = StellariumRPC()

In [32]:
location = EarthLocation(lat=33.30167*u.deg, lon=-87.60750*u.deg, height=85*u.m)

In [34]:
R = np.identity(3)

In [35]:
def get_true():
    ra, dec = rpc.get_selected_ra_dec()
    while ra < 0:
        ra += 360
    return ra, dec

In [36]:
def J2000_to_AltAz(ra, dec):
    ra *= 24/360
    return equatorial_to_horizontal(ra, dec, location=location, 
                                    pressure=1013*u.hPa, temperature=15*u.Celsius, rel_humidity=0.5)

In [41]:
def AltAz_to_TA(azi, alt):
    """
    Need a known rotation matrix.
    """
    v2 = vec_from_angles(alt, azi)
    v1 = np.squeeze(np.asarray(np.matmul(R.T, v2)))
    return angles_from_vec(v1)


In [45]:
def TA_to_cnts(phi, theta):
    """
    Need mount errors and encoder/gearing stuff
    """
    phi_npr   = 15000
    theta_npr = 26400
    
    phi_cnt = int(phi*phi_npr/360)
    theta_cnt = int(theta*theta_npr/360)
    
    return phi_cnt, theta_cnt
    

In [46]:
ra_true, dec_true = get_true()
azi, alt = J2000_to_AltAz(ra_true, dec_true)
theta, phi = AltAz_to_TA(azi, alt)
pcnt, tcnt = TA_to_cnts(phi, theta)

print("RA/Dec:    {:7.3f}, {:7.3f}".format(ra_true, dec_true))
print("Azi/Alt:   {:7.3f}, {:7.3f}".format(azi, alt))
print("Phi/Theta: {:7.3f}, {:7.3f}".format(phi, theta))
print("Pcnt/Tcnt: {:7d}, {:7d}".format(pcnt, tcnt))

RA/Dec:    279.429,  38.805
Azi/Alt:   291.260,  62.398
Phi/Theta: 291.260,  62.398
Pcnt/Tcnt:   12135,    4575
